In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

In [ ]:
!pip install transformers timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [ ]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoProcessor, Blip2ForConditionalGeneration, BitsAndBytesConfig
from PIL import Image
import pandas as pd
import os
from tqdm import tqdm
from io import BytesIO

In [ ]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!pip install -q accelerate bitsandbytes datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch

# Use GPU if available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
# processor = BlipProcessor.from_pretrained("Salesforce/blip2-image-captioning-base")
# model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip2-image-captioning-base")
# Set up 8-bit quantization config
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b",device_map="auto",  quantization_config=quantization_config)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/6.96k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/5.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate_caption(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    outputs = model.generate(**inputs)
    caption = processor.decode(outputs[0], skip_special_tokens=True)
    return caption


In [ ]:
# Initialize a counter
file_count = 0

image_folder = '/content/mydrive/MyDrive/Image Folder 2'
captions = {}
corrupted_files = []

for filename in os.listdir(image_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder, filename)
        try:
          caption = generate_caption(image_path)
          captions[filename] = caption
          # Increment and print the counter
          file_count += 1
          print(f"Processed {file_count} files")
          # print(f"Image: {filename}, Caption: {caption}")
        except OSError as e:
            # Log the corrupted file and continue with the next
            print(f"Skipping corrupted file: {filename}")
            corrupted_files.append(filename)


Processed 1 files
Processed 2 files
Processed 3 files
Processed 4 files
Processed 5 files
Processed 6 files
Processed 7 files
Processed 8 files
Processed 9 files
Processed 10 files
Processed 11 files
Processed 12 files
Processed 13 files
Processed 14 files
Processed 15 files
Processed 16 files
Processed 17 files
Processed 18 files
Processed 19 files
Processed 20 files
Processed 21 files
Processed 22 files
Processed 23 files
Processed 24 files
Processed 25 files
Processed 26 files
Processed 27 files
Processed 28 files
Processed 29 files
Processed 30 files
Processed 31 files
Processed 32 files
Processed 33 files
Processed 34 files
Processed 35 files
Processed 36 files
Processed 37 files
Processed 38 files
Processed 39 files
Processed 40 files
Processed 41 files
Processed 42 files
Processed 43 files
Processed 44 files
Processed 45 files
Processed 46 files
Processed 47 files
Processed 48 files
Processed 49 files
Processed 50 files
Processed 51 files
Processed 52 files
Processed 53 files
Pr

In [ ]:
csv_file_path = '/content/mydrive/MyDrive/Colab Notebooks/evaluation/Image_folder_2.csv'
captions_df = pd.DataFrame(list(captions.items()), columns=['image_name', 'Caption'])
captions_df.to_csv(csv_file_path, index=False)

if corrupted_files:
    print(f"The following files were corrupted and skipped: {corrupted_files}")

print(f"Captions saved to {csv_file_path}")

Captions saved to /content/mydrive/MyDrive/Colab Notebooks/Image_folder_2.csv
